Generating large amounts of pseudorandom numbers based on PRNG

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator
import xml.etree.ElementTree as Xet

In [3]:
def lfsr(seed, taps, length):
    """Generate a LFSR sequence of a given length."""
    # Convert seed to binary representation with 32 bits
    register = np.array([int(bit) for bit in f"{seed:032b}"], dtype=np.uint8)
    output = np.empty(length, dtype=np.uint8)
    
    for i in range(length):
        new_bit = np.bitwise_xor.reduce(register[taps])
        # Shift the register and insert the new bit at the end
        register[:-1] = register[1:]
        register[-1] = new_bit
        # Save output bit
        output[i] = register[0]
    
    return output
    
def insert_sequence_into_db(cursor, taps, sequence):
    """Inserts the generated sequence into the database."""
    # Convert the sequence from np.array to a string representation of binary numbers
    binary_str = ''.join(map(str, sequence))
    taps_str = ','.join(map(str, taps))  # Convert taps list to string
    cursor.execute('''
        INSERT INTO kissat_data (taps, output_binary)
        VALUES (?, ?)
    ''', (taps_str, int(binary_str, 2)))  # Convert binary string to integer before storing


In [4]:
import sqlite3
#   Uncomment this to create your own local database, ideally with a different name for each solver
"""
conn = sqlite3.connect('prng_data.db')
cursor = conn.cursor()

# Also change here!
cursor.execute('''
    CREATE TABLE IF NOT EXISTS kissat_data ( 
        id INTEGER PRIMARY KEY,
        taps TEXT,
        output_binary INTEGER
    )
''')

# Close the connection
conn.close()
"""


In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('prng_data.db')
cursor = conn.cursor()

# Parameters for LFSR
seeds = [1]  # You can add more seeds if you want to generate more sequences
taps_list = [[31, 30, 1, 0],]  # First is what we use, second is XORMIX (soon)
length = 1000  # Length of the sequence to generate

# Generate and insert sequences into the database
for seed in seeds:
    for taps in taps_list:
        sequence = lfsr(seed, taps, length)
        insert_sequence_into_db(cursor, taps, sequence)

# Commit changes and close the connection
conn.commit()
conn.close()